In [ ]:
1

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from openfisca_france import CountryTaxBenefitSystem

In [ ]:
import sys

sys.path.append("../technique")

In [ ]:
from dotenv import load_dotenv

load_dotenv()
import os

In [ ]:
from utils import *

In [ ]:
from mobilite import add_compensation, build_data, compute_result, get_df, get_results

In [ ]:
base = CountryTaxBenefitSystem()
base.load_extension("openfisca_france_local")

In [ ]:
qf_fiscal_reform = QfFiscalReform(base)

In [ ]:
statut_reform = StatutReform(qf_fiscal_reform)

In [ ]:
reform = statut_reform

In [ ]:
pd.options.display.float_format = lambda v: "{:_.2f}".format(v).replace("_", " ")

In [ ]:
get_results(base, sample_count=3, reform=reform)

In [ ]:
df, compens_constant = get_df()

In [ ]:
rdf = pd.DataFrame(
    data={
        "pu_fichier": df.PU,
        "pu_calc": df.PU * 0,
        "tp": df.PU * 0,
    }
)
data, complement_df = build_data(df, rdf, sample_count=3)
scenario = StrasbourgSurveyScenario(base, data=data)
(res, recettes) = compute_result(scenario, complement_df, 0, compens_constant)

In [ ]:
r_scenario = StrasbourgSurveyScenario(reform, data=data)
(r_res, r_recettes) = compute_result(r_scenario, complement_df, 0, compens_constant)

In [ ]:
pd.options.display.float_format = lambda v: "{:_.2f}".format(v).replace("_", " ")

In [ ]:
pd.DataFrame(res.groupby(["sample_id"]).pu_calc_ht.sum().describe()).transpose()

In [ ]:
res[~res.ok].titre_fichier.unique()

In [ ]:
assert [*res[~res.ok].titre_fichier.unique()] == [" +65 Annuel", "PMR Gratuit"] or [
    *res[~res.ok].titre_fichier.unique()
] == [
    " +65 Annuel",
    "19-25 annuel",
    "26-64 annuel",
    "Annuel 4-18",
    "Annuel PMR",
    "PMR Gratuit",
]

In [ ]:
stat_df = pd.DataFrame(
    {
        "sample_id": res.sample_id,
        "individu_id": res.individu_id,
        "count": 1,
        "CAF": data["input_data_frame_by_entity"]["famille"]["qf_caf"],
        "EMS": data["input_data_frame_by_entity"]["famille"]["qf_fiscal"],
        "input": res.pu_calc,
    }
)
stat_df
res_s = stat_df[stat_df.sample_id == 0]

In [ ]:
counts = res_s.groupby("input").EMS.count()
print(counts)
if sum(counts.index == 0):
    pre_sum = counts.sum()
    assert sum(counts.index == 0) == 1
    assert counts.index[0] == 0
    counts[counts.index == counts.index[counts.index != 0].min()] += counts[0]
    counts[0] = 0
    assert counts.sum() == pre_sum
    print(counts)

In [ ]:
bracket_limits = counts.cumsum()
bracket_limits

In [ ]:
ranks = res_s.EMS.rank(method="first")
ranks

In [ ]:
indexes = ranks.isin(bracket_limits)
rr_res = pd.DataFrame(
    data={
        "rank": ranks[indexes],
        "value": res_s.EMS[indexes],
    }
)

In [ ]:
(res_s.EMS == 0).sum() / (res_s.EMS == 0).count()

In [ ]:
import seaborn

In [ ]:
seaborn.ecdfplot(stat_df[["CAF", "EMS"]])

In [ ]:
rr_res

In [ ]:
if False:
    count = df.quantité.sum().astype("int64")
    static_sample_count = 3
    static_sample_ids = np.repeat(list(range(static_sample_count)), count)
    static_sample_individu_df = pd.DataFrame(
        {
            "sample_id": static_sample_ids,
            "famille_id": list(range(count * static_sample_count)),
            "qfrule": np.tile(np.repeat(df.QF, df.quantité), static_sample_count),
            "agerule": np.tile(np.repeat(df.AGE, df.quantité), static_sample_count),
            "taux_incapacite": np.tile(
                np.repeat(np.where(df.Titres.str.contains("PMR"), 0.8, 0), df.quantité),
                static_sample_count,
            ),
        }
    )

    determine_qf(static_sample_individu_df)

    rr = static_sample_individu_df.groupby(by="sample_id").qf_fiscal.rank(
        method="first"
    )

    df[["QF", "quantité"]].groupby(by="QF").sum()

    indexes = rr.isin(df[["QF", "quantité"]].groupby(by="QF").sum().cumsum().quantité)
    rr_res = pd.DataFrame(
        data={
            "rank": rr[indexes],
            "value": static_sample_individu_df.qf_fiscal[indexes],
        }
    )

    rr_res.groupby(by="rank").describe()

    no_reduit_indexes = rr.isin(merge_bareme_population)
    no_reduit_rr_res = pd.DataFrame(
        data={
            "rank": rr[no_reduit_indexes],
            "value": static_sample_individu_df.qf_fiscal[no_reduit_indexes],
        }
    )

    no_reduit_rr_res_desc = no_reduit_rr_res.groupby(by="rank").value.describe()
    no_reduit_rr_res_desc

    no_reduit_rr_res_desc["50%"].round().astype("int64")